In [1]:
# paibox 和 spikingjelly 的推理结果比较，输出为 csv 文件
# 需要修改数据集路径DVS128GESTURE_DATA_DIR
import torch
import numpy as np
import paibox as pb
import csv

# PAIBox网络定义
class Conv2d_Net(pb.Network):
    def __init__(self, channels, param_dict):
        super().__init__()

        self.i0 = pb.InputProj(input=None, shape_out=(2, 69, 52))
        self.n0 = pb.LIF((channels*2, 35, 26), bias=param_dict['conv.0.bias'], threshold=param_dict['conv.0.vthr'], reset_v=0, tick_wait_start=1) # convpool3x3
        self.conv2d_0 = pb.Conv2d(self.i0, self.n0, kernel=param_dict['conv.0.weight'], padding=2, stride=2)

        self.n1 = pb.LIF((channels*2, 35, 26), bias=param_dict['conv.2.bias'], threshold=param_dict['conv.2.vthr'], reset_v=0, tick_wait_start=2) # conv3x3
        self.conv2d_1 = pb.Conv2d(self.n0, self.n1, kernel=param_dict['conv.2.weight'], padding=2, stride=1)

        self.n2 = pb.LIF((channels*4, 18, 13), bias=param_dict['conv.4.bias'], threshold=param_dict['conv.4.vthr'], reset_v=0, tick_wait_start=3) # convpool3x3
        self.conv2d_2 = pb.Conv2d(self.n1, self.n2, kernel=param_dict['conv.4.weight'], padding=2, stride=2)

        self.n3 = pb.LIF((channels*4, 18, 13), bias=param_dict['conv.6.bias'], threshold=param_dict['conv.6.vthr'], reset_v=0, tick_wait_start=4) # conv3x3
        self.conv2d_3 = pb.Conv2d(self.n2, self.n3, kernel=param_dict['conv.6.weight'], padding=2, stride=1)

        # self.n4 = pb.LIF((channels*8, 8, 8), bias=param_dict['conv.8.bias'], threshold=param_dict['conv.8.vthr'], reset_v=0, tick_wait_start=5) # convpool3x3
        # self.conv2d_4 = pb.Conv2d(self.n3, self.n4, kernel=param_dict['conv.8.weight'], padding=2, stride=2)

        # self.n5 = pb.LIF((channels*8, 8, 8), bias=param_dict['conv.10.bias'], threshold=param_dict['conv.10.vthr'], reset_v=0, tick_wait_start=6) # conv3x3
        # self.conv2d_5 = pb.Conv2d(self.n4, self.n5, kernel=param_dict['conv.10.weight'], padding=2, stride=1)

        self.n10 = pb.LIF(channels*8 * 8 * 8, threshold=param_dict['fc.2.vthr'], reset_v=0, tick_wait_start=5) # fc
        self.fc_0 = pb.FullConn(self.n3, self.n10, conn_type=pb.SynConnType.All2All, weights=param_dict['fc.2.weight'])

        self.n11 = pb.LIF(channels*8 * 4 * 4, threshold=param_dict['fc.5.vthr'], reset_v=0, tick_wait_start=6) # fc
        self.fc_1 = pb.FullConn(self.n10, self.n11, conn_type=pb.SynConnType.All2All, weights=param_dict['fc.5.weight'])

        self.n12 = pb.LIF(80, threshold=param_dict['fc.8.vthr'], reset_v=0, tick_wait_start=7) # fc
        self.fc_2 = pb.FullConn(self.n11, self.n12, conn_type=pb.SynConnType.All2All, weights=param_dict['fc.8.weight'])

        self.probe1 = pb.Probe(self.n12, "spike")

C = 2

# PAIBox网络初始化
param_dict = {}
def getNetParam():
    timestep = 8
    layer_num = 9
    delay = layer_num - 1
    param_dict["timestep"] = timestep
    param_dict["layer_num"] = layer_num
    param_dict["delay"] = delay

    param_dict['conv.0.weight']=np.random.randint(1,127,size=(C*2,2,5,5))
    param_dict['conv.0.bias']=np.random.randint(1,127,size=(C*2))
    param_dict['conv.2.weight']=np.random.randint(1,127,size=(C*2,C*2,5,5))
    param_dict['conv.2.bias']=np.random.randint(1,127,size=(C*2))
    param_dict['conv.4.weight']=np.random.randint(1,127,size=(C*4,C*2,5,5))
    param_dict['conv.4.bias']=np.random.randint(1,127,size=(C*4))
    param_dict['conv.6.weight']=np.random.randint(1,127,size=(C*4,C*4,5,5))
    param_dict['conv.6.bias']=np.random.randint(1,127,size=(C*4))
    param_dict['conv.8.weight']=np.random.randint(1,127,size=(C*8,C*4,5,5))
    param_dict['conv.8.bias']=np.random.randint(1,127,size=(C*8))
    param_dict['conv.10.weight']=np.random.randint(1,127,size=(C*8,C*8,5,5))
    param_dict['conv.10.bias']=np.random.randint(1,127,size=(C*8))
    param_dict['fc.2.weight']=np.random.randint(1,127,size=(C*8*8*8,C*4*18*13)).T
    param_dict['fc.5.weight']=np.random.randint(1,127,size=(C*8*4*4,C*8*8*8)).T
    param_dict['fc.8.weight']=np.random.randint(1,127,size=(80,C*8*4*4)).T
    param_dict['conv.0.vthr']=10
    param_dict['conv.2.vthr']=20
    param_dict['conv.4.vthr']=30
    param_dict['conv.6.vthr']=40
    param_dict['conv.8.vthr']=50
    param_dict['conv.10.vthr']=60
    param_dict['fc.2.vthr']=70
    param_dict['fc.5.vthr']=80
    param_dict['fc.8.vthr']=90
getNetParam()

# PAIBox仿真器
pb_net = Conv2d_Net(C, param_dict)
sim = pb.Simulator(pb_net)


# test(test_num=50)
pb.BACKEND_CONFIG.target_chip_addr = (0, 0)

# print(pb_net.conv2d_0.name)
# print(type(pb_net.n0))
# print(pb_net.n0.shape_in)
# print(pb_net.n0.shape_out)
# print(type(pb_net.n0[0,:,:]))
# print(pb_net.n0[0,:,:].shape_in)
# print(pb_net.n0[0,:,:].shape_out)
# n1_0_1 = (pb_net.n11,pb_net.n11)
# print(type(n1_0_1))

c:\Users\oilgi\AppData\Local\Programs\Python\Python310\lib\site-packages\paibox\components\synapses\transforms.py:103: AutoOptimizationWarning: dtype of weight is optimized automatically, int32 -> int8.
  warnings.warn(


In [3]:
mapper = pb.Mapper()

mapper.build(pb_net)

# Core estimate only
graph_info = mapper.compile(
    weight_bit_optimization=True, grouping_optim_target="both", core_estimate_only=True
)

# #N of cores required
print("Core required:", graph_info["n_core_required"])

# Clear all the results
mapper.clear()

Core required: 897


In [3]:
mapper = pb.Mapper()

mapper.build(pb_net)

graph_info = mapper.compile(
    weight_bit_optimization=True, grouping_optim_target="both"
)

# #N of cores required
print("Core required:", graph_info["n_core_required"])

Core required: 897


In [ ]:
mapper.export(
    write_to_file=True, fp="./debug", format="npy", export_core_params=False
)

# Clear all the results
mapper.clear()